In [2]:
import random
import time

# Define the factors
population_size = 300
mutation_rate = 0.1
crossover_probability = 0.9
num_generations = 300

# Sudoku constraints
all_letters = ['W', 'O', 'R', 'D']

def fitness(solution):
    fitness_value = 0

    # Check rows for duplicates
    valid_rows = [set(solution[i*4:i*4+4]) == set(all_letters) for i in range(4)]

    # Check columns for duplicates
    valid_cols = [set(solution[i::4]) == set(all_letters) for i in range(4)]

    # Check each 2x2 sub-grid for duplicates
    sub_grids = [
        [0, 1, 4, 5], [2, 3, 6, 7],
        [8, 9, 12, 13], [10, 11, 14, 15]
    ]

    valid_sub_grids = [set(solution[idx] for idx in sub_grid) == set(all_letters) for sub_grid in sub_grids]

    # Convert boolean values to 1 (True) or 0 (False) before summing
    fitness_value += sum(int(valid) for valid in valid_rows)
    fitness_value += sum(int(valid) for valid in valid_cols)
    fitness_value += sum(int(valid) for valid in valid_sub_grids)

    return fitness_value

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 2)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutation(individual):
    mutation_point = random.randint(0, len(individual)-1)
    individual[mutation_point] = random.choice(all_letters)
    return individual

def tournament_selection(population, tournament_size=3):
    tournament = random.sample(population, tournament_size)
    winner = max(tournament, key=lambda x: fitness(x))
    return winner

# Define partially completed grid
partial_grid = [
    ['W', 'O', 'R', None],
    [None, 'D', None, None],
    [None, None, None, None],
    ['D', None, None, None]
]

# Initialize the population
initial_population = []
for _ in range(population_size):
    individual = []
    for i in range(4):
        for j in range(4):
            if partial_grid[i][j] is not None:
                individual.append(partial_grid[i][j])
            else:
                individual.append(random.choice(all_letters))
    initial_population.append(individual)

population = initial_population

# Evolution loop
tic = time.time()
for generation in range(num_generations):
    new_population = []

    for _ in range(population_size // 2):
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)

        if random.random() < crossover_probability:
            child1, child2 = crossover(parent1, parent2)  # Include parameters in the function call

            if random.random() < mutation_rate:
                child1 = mutation(child1)
            if random.random() < mutation_rate:
                child2 = mutation(child2)

            new_population.extend([child1, child2])

    population = new_population
toc = time.time()

# Final solution
best_solution = max(population, key=lambda x: fitness(x))

print("time taken: ", toc - tic)
print("fitness: ", fitness(best_solution))
print("The best solution: ")
for i in range(4):
    print(' '.join(best_solution[i*4:i*4+4]))

time taken:  4.971381664276123
fitness:  12
The best solution: 
W O R D
R D W O
O R D W
D W O R
